In [1]:
import (
	"database/sql"
	"fmt"
	"net/url"

	_ "github.com/lib/pq"
)

In [2]:
func ResultSet(sqlStmt *string) ([][]string, error) {
	var err error
	var results *sql.Rows

    conn, err := sql.Open(
        "postgres", 
        "user=admin password=admin host=deployments_virtual_data_port_1 port=9996 sslmode=disable database=",
    )
    if err != nil {
        fmt.Println(err)
    }
    
	results, err = conn.Query(*sqlStmt)
	if err != nil {
		return nil, err
	}

	columns, err := results.Columns()
	if err != nil {
		return nil, err
	}

	count := len(columns)
	tableData := [][]string{}
	values := make([]interface{}, count)
	valuePtrs := make([]interface{}, count)

	for results.Next() {
		for i := 0; i < count; i++ {
			valuePtrs[i] = &values[i]
		}
		results.Scan(valuePtrs...)
		record := make([]string, count)
		for i := range columns {
			switch values[i].(type) {
			case nil:
				var s string
				record[i] = s
			case []byte:
				record[i] = string(values[i].([]byte))
			default:
				record[i] = fmt.Sprintf("%s", values[i])
			}
		}
		tableData = append(tableData, record)
	}

	return tableData, nil
}

In [12]:
sqlStmt := `SELECT
  internal_id,
  database_name,
  name,
  user_creator,
  last_user_modifier,
  create_date,
  last_modification_date,
  description,
  folder
FROM GET_ELEMENTS()
WHERE name = 'test_123'
  AND database_name = 'ds_postgres'
  AND type = 'view'
  AND subtype = 'base';`
resultSet, err := ResultSet(&sqlStmt)
if err != nil {
    fmt.Println(err)
}
resultSet[0]

[19d33cb1-0197-4996-a548-0b7992400743 ds_postgres test_123 admin admin 2021-11-28 21:37:08 -0800 -0800 2021-11-28 21:37:08 -0800 -0800  /]

In [28]:
sqlStmt := `CONNECT DATABASE ds_postgres;
CALL GENERATE_VQL_TO_CREATE_JDBC_BASE_VIEW(
	'ds_postgres',
	NULL,
	NULL,
	'test_123',
	'bv_test_123',
	NULL,
	'ds_postgres'
);`
resultSet, err := ResultSet(&sqlStmt)
if err != nil {
    fmt.Println(err)
}

In [39]:
resultSet[1][0].(string)

ERROR: repl.go:1:18: invalid type assertion: resultSet[1][0].(string) (non-interface type <string> on left)

In [37]:
var v string
if v {
    fmt.Println("hi")
}

ERROR: repl.go:2:4: invalid boolean predicate, expecting <bool> expression, found <string>: v